In [8]:
class Cola:
    def __init__(self):
        self.citasProcesar = []
        self.tamanio = 0
    def agregarCita(self, cita):
        self.citasProcesar.append(cita)
        print("CITA AGREGADA EXITOSAMENTE")
    def dequeCita(self):
        if self.tamanio == 0:
            return False
        else:
            return self.citasProcesar.pop(0)
    def front(self):
        if self.tamanio == 0:
            return None
        else:
            return self.citasProcesar[0]
    def recorrerCola(self):
        if self.tamanio == 0:
            return "NO HAY CITAS REGISTRADAS EN LA COLA"
        else:
            turno = 1
            for cita in self.citasProcesar:
                print(f'TURNO: {turno}, CITA: {cita}')
                turno += 1

In [9]:
class Nodo:
    def __init__(self, valor):
        self.valor = valor
        self.siguiente = None
    def __str__(self):
        return f'Valor: {self.valor}'

In [10]:
class ListaEnlazada:
	def __init__(self):
		self.nodoCabeza = None
		self.tamanio = 0
	def push_back(self,valor):
		nuevoNodo = Nodo(valor)
		if self.nodoCabeza == None:
			self.nodoCabeza = nuevoNodo
			self.tamanio += 1
		else:
			nodoActual = self.nodoCabeza
			while nodoActual.siguiente != None:
				nodoActual = nodoActual.siguiente
				nodoActual.siguiente = nuevoNodo
				self.tamanio += 1
	def recorrerLista(self):
		if self.nodoCabeza != None:
			nodoActual = self.nodoCabeza
			while nodoActual != None:
				print(nodoActual,end="--->")
				nodoActual = nodoActual.siguiente
	def deleteNodo(self, pos):
		if self.nodoCabeza == None:
			return False
		if pos > self.tamanio:
			return False
		if pos == 0 and self.tamanio == 1:
			self.nodoCabeza = None
		elif pos == 0:
			siguiente = self.nodoCabeza.siguiente
			self.nodoCabeza = siguiente
			self.tamanio -= 1
		else:
			nodoPrevio = self.nodoCabeza
			nodoSiguiente = self.nodoCabeza.siguiente
			for i in range(pos-1):
				nodoPrevio = nodoSiguiente
				nodoSiguiente = nodoSiguiente.siguiente
				if(nodoSiguiente == None):
					nodoPrevio.siguiente = None
				else:
					nodoPrevio.siguiente = nodoSiguiente.siguiente
					self.tamanio -= 1

In [11]:
class Cita:
    id_counter = 1
    
    def __init__(self, nombrePaciente, hora, nombreDoctor, estado):
        self.id = Cita.id_counter
        Cita.id_counter += 1
        self.nombrePaciente = nombrePaciente
        self.hora = hora
        self.nombreDoctor = nombreDoctor
        self.estado = estado
        
    def __str__(self):
        return f"ID: {self.id} | Paciente: {self.nombrePaciente} | Hora: {self.hora} | Doctor: {self.nombreDoctor} | Estado: {self.estado}"
    

In [12]:
cola_pendientes = Cola()
lista_completa = ListaEnlazada()
def registrar_cita():
    paciente = input("Nombre del paciente: ")
    hora = input("Hora de la cita: ")
    doctor = input("Nombre del doctor: ")
    nueva_cita = Cita(paciente, hora, doctor, "Programada")
    cola_pendientes.agregarCita(nueva_cita)
    lista_completa.push_back(nueva_cita)
    print(f"Cita registrada con ID: {nueva_cita.id}")
historial_modificaciones = []
def modificar_cita():
    id_input = input("Ingrese el ID de la cita a modificar: ")
    if not id_input.isdigit():
        print("Error: El ID debe ser un número")
        return
    
    id_buscar = int(id_input)
    nodo_actual = lista_completa.nodoCabeza
    encontrado = False
    
    while nodo_actual:
        if nodo_actual.valor.id == id_buscar:
            cita = nodo_actual.valor
            print("\nCita encontrada:")
            print(cita)
            copia = Cita(cita.nombrePaciente, cita.hora, cita.nombreDoctor, cita.estado)
            copia.id = cita.id
            historial_modificaciones.append(copia)
            print("\nOpciones de modificación:")
            print("1. Nombre del paciente")
            print("2. Hora")
            print("3. Doctor")
            print("4. Estado")
            opcion = input("Elija qué modificar (1-4): ")
            if opcion == "1":
                cita.nombrePaciente = input("Nuevo nombre: ")
            elif opcion == "2":
                cita.hora = input("Nueva hora: ")
            elif opcion == "3":
                cita.nombreDoctor = input("Nuevo doctor: ")
            elif opcion == "4":
                cita.estado = input("Nuevo estado: ")
            else:
                print("Opción no válida")
                return
            print("¡Cambios guardados!")
            encontrado = True
            break  
        nodo_actual = nodo_actual.siguiente 
    if not encontrado:
        print("No existe una cita con ese ID")
print('Sistema de Gestión de Citas Médicas')
while True:
    print('\n1. Registrar cita\n2. Listar citas\n3. Modificar cita\n4. Salir')
    opcion = input("Seleccione una opción: ")
    
    if opcion == "1":
        registrar_cita()
    # elif opcion == "2":
        #mostrar_citas() su codigo olave
    elif opcion == "3":
        modificar_cita()
    elif opcion == "4":
        print("Saliendo del sistema...")
        break
    else:
        print("Opción inválida, intente nuevamente")


Sistema de Gestión de Citas Médicas

1. Registrar cita
2. Listar citas
3. Modificar cita
4. Salir
CITA AGREGADA EXITOSAMENTE
Cita registrada con ID: 1

1. Registrar cita
2. Listar citas
3. Modificar cita
4. Salir
CITA AGREGADA EXITOSAMENTE
Cita registrada con ID: 2

1. Registrar cita
2. Listar citas
3. Modificar cita
4. Salir
Saliendo del sistema...
